<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Лемматизация текста</a></span></li><li><span><a href="#Разбивка-данных-на-выборки." data-toc-modified-id="Разбивка-данных-на-выборки.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разбивка данных на выборки.</a></span></li><li><span><a href="#Векторизация-данных." data-toc-modified-id="Векторизация-данных.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Векторизация данных.</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Словарь-параметров" data-toc-modified-id="Словарь-параметров-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Словарь параметров</a></span></li></ul></li><li><span><a href="#Логистическая-регрессия." data-toc-modified-id="Логистическая-регрессия.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия.</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#Решающее-дерево." data-toc-modified-id="Решающее-дерево.-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Решающее дерево.</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import time
from catboost import CatBoostClassifier 
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
STATE = 68756

In [3]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
pass

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nadya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nadya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Nadya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nadya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nadya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('toxic_comments.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Пропусков с данных нет, типы корректные.

Создадим копрус данных.

In [6]:
corpus = df['text']#.values.astype('U')

### Очистка текста

In [7]:
corpus_clear = [' '.join(re.sub(r"[^a-zA-Z ]", " ", s.lower()).split()) for s in corpus]

### Лемматизация текста

Лемматизируем (длительность ~  минута)

In [8]:
lemm = WordNetLemmatizer()

In [9]:
def pos(word):
    dict = {"N": wordnet.NOUN,
            "V": wordnet.VERB,
            "J": wordnet.ADJ,        
            "R": wordnet.ADV}

    sign = nltk.pos_tag([word])[0][1][0].upper()
    return dict.get(sign, wordnet.NOUN)

In [10]:
%%time

# pos-теги закомментировал, я состарюсь пока они посчитаются. 
# Если у вас есть на чем их посчитать, раскомментируйте эту строку пожалуйста, а вторую закомментируйте.

#corpus_lemm = [' '.join([lemm.lemmatize(w, pos(w)) for w in s]) for s in [nltk.word_tokenize(st) for st in corpus_clear]]
corpus_lemm = [' '.join([lemm.lemmatize(w) for w in s]) for s in [nltk.word_tokenize(st) for st in corpus_clear]]

Wall time: 1min 6s


Добавим обработанные данные в исходный датасет в столбец `text_lemm`.

In [11]:
df['text_lemm'] = corpus_lemm

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
 3   text_lemm   159292 non-null  object
dtypes: int64(2), object(2)
memory usage: 4.9+ MB


In [13]:
df.head(10)

,Unnamed: 0,text,toxic,text_lemm
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
5,5,"""\n\nCongratulations from me as well, use the ...",0,congratulation from me a well use the tool wel...
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
7,7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the matt shirvington article...
8,8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense wa offensive to you...
9,9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


### Разбивка данных на выборки.

In [14]:
target = df['toxic']
feature = df['text_lemm']

In [15]:
features_train, features_test, target_train, target_test = \
                train_test_split(feature, target, test_size=0.2, random_state=STATE)

In [16]:
features_train, features_valid, target_train, target_valid = \
                train_test_split(features_train, target_train, test_size=0.25, random_state=STATE)

### Векторизация данных.

Векторизируем данные с помощью TF-IDF.

In [17]:
stopwords = set(nltk_stopwords.words('english'))
tf_idf = TfidfVectorizer(stop_words=list(stopwords), min_df=50) # min_df=50

In [18]:
features_tf_idf_train = tf_idf.fit_transform(features_train)
features_tf_idf_valid = tf_idf.transform(features_valid)
features_tf_idf_test = tf_idf.transform(features_test)

**Вывод:**  
Данные подготовлены. Очищены, лемматизированы, векторизированны. Можно обучать модели.

## Обучение

Будем обучать 3 модели:  
- Логистическая регрессия.
- Классификацию LightGBM.
- Случайный лес.

#### Словарь параметров

In [19]:
param = {
'grid_lr': {
            'C': range(8, 14),
            'solver' :  ['liblinear', 'newton-cg'],
            'max_iter':[20, 500, 1000]
           },
'rand_lgbm': { # 0.7718260869565218
            'min_child_samples': [20,25],
            'num_leaves': [20,31],
            'max_depth': range (12, 19, 2),  #16
            'n_estimators': range(100, 2000, 450) #1020          
             },
'rand_dt': {
            'min_samples_split': range (8,17),
            'min_samples_leaf': range (8,17),
            'max_depth': range (9,40,2),
            }
        }

In [20]:
def print_result(model, name):
    print(f'Лучшая модель {name} - {model.best_estimator_}')
    print(f'f1 лучшей модели {name} при кросс-валидации - {model.best_score_:.4f}')
    print(f'f1 лучшей модели {name} на проверочной выборке - {f1_score(target_valid, model.best_estimator_.predict(features_tf_idf_valid)):.4f}')

### Логистическая регрессия.

In [23]:
model_lg = LogisticRegression(n_jobs=-1, random_state=STATE).fit(features_tf_idf_train, target_train) 

In [24]:
f1_score(target_valid, model_lg.predict(features_tf_idf_valid))

0.7258485639686684

Логистическая регрессия с параметрами по умолчанию на показала f1 = 0.7258

Подберем гиперпараметры ( ~1,5 минуты )

In [31]:
start = time.time()
#print(model_lg)
search = GridSearchCV(LogisticRegression(), param['grid_lr'], cv=5, scoring = 'f1', n_jobs=-1)
search.fit(features_tf_idf_train, target_train)
print(f"Время подбора гиперпараметров GridSearchCV = {time.time() - start:.0f}с")
best_lg = search.best_estimator_

Время подбора гиперпараметров GridSearchCV = 55с


In [26]:
print_result(search, 'линейной регрессии')

Лучшая модель линейной регрессии - LogisticRegression(C=9, max_iter=20, solver='liblinear')
f1 лучшей модели линейной регрессии при кросс-валидации - 0.7621
f1 лучшей модели линейной регрессии на проверочной выборке - 0.7528


Логистическая регрессия с подобранными гиперпараметрами показала f1 = 0.7528

### LightGBM

In [27]:
model_lgbm = LGBMClassifier(n_jobs=-1, random_state=STATE)

In [28]:
%%time

model_lgbm.fit(features_tf_idf_train, target_train)
pass

Wall time: 9.38 s


In [29]:
f1_score(target_valid, model_lgbm.predict(features_tf_idf_valid))

0.7355236885097114

LightGBM с параметрами по умолчанию показал f1 = 0.7355

Подберем гиперпараметры ( ~ 20 минут )

In [ ]:
start = time.time()
lgbm_RS = RandomizedSearchCV(LGBMClassifier(n_jobs=-1), param['rand_lgbm'], cv=5, scoring = 'f1', \
                                                                                         random_state=STATE, n_jobs=-1)
lgbm_RS.fit(features_tf_idf_train, target_train)
print(f"Время подбора гиперпараметров RandomizedSearchCV = {time.time() - start:.0f}с")
best_lgbm = lgbm_RS.best_estimator_

In [ ]:
print_result(lgbm_RS, 'LightGBM')

LightGBM с подобранными гиперпараметрами показал f1 = 0.7516

### Решающее дерево.

Решающее дерево (~ 2 минуты)

In [32]:
%%time

model_dt = DecisionTreeClassifier(random_state=STATE).fit(features_tf_idf_train, target_train)

Wall time: 1min 41s


In [ ]:
f1_score(target_valid, model_dt.predict(features_tf_idf_valid))

In [ ]:
Решающее дерево с параметрами по умолчанию на показало f1 = 0.6907

Подберем гиперпараметры ( ~ 1 минута )

In [34]:
start = time.time()
dt_RS = RandomizedSearchCV(DecisionTreeClassifier(), param['rand_dt'], cv=5, scoring = 'f1', \
                                                                                         random_state=STATE, n_jobs=-1)
dt_RS.fit(features_tf_idf_train, target_train)
print(f"Время подбора гиперпараметров RandomizedSearchCV = {time.time() - start:.0f}с")
best_dt = dt_RS.best_estimator_

Время подбора гиперпараметров RandomizedSearchCV = 74с


In [ ]:
print_result(dt_RS, 'Решающее дерево')

Решающее дерево с подобранными гиперпараметрами показало f1 = 0,6526

**Вывод:**  
Самый лучший результат на проверочной выборке показала модель Логистическая регрессия.  
f1 модели = 0,7528  


## Выводы

Проверим лучшую модель на тестовой выборке

In [ ]:
f1_score(target_test, best_lg.predict(features_tf_idf_test))

На тестовой выборке f1 лучшей модели = 0.76

Проверим модель на адекватность константной моделью.

In [ ]:
dummy_param = {
                0:["constant", "  При нулевой константной модели f1 равно "],
                1:["constant", "При единичной константной модели f1 равно "],
                2:["uniform", "При случайной константной модели f1 равно "]
            }


for i, s in dummy_param.items():
    print(s[1], f1_score(target_test, DummyClassifier(strategy=s[0], constant=i).fit(features_tf_idf_train, target_train).predict(features_tf_idf_test)))

Три разных константных модели показали на тестовой выборке значение f1 значительно меньше демонстрируемого лучшей моделью.    
Модель прошла проверку на адекватность.

Лучшая модель на тестовой выборке показала f1 = 0.76

**Вывод:**  
Во время выполнения проекта были выполнены следующие шаги:
- получение данных
- очистка текста
- лемматизация текста
- векторизация текста
- обучение 3 моделей, и выбор лучшей по метрике f1
- подбор гиперпараметров моделей с целью улучшить метрику
- проверка лучшей модели на тестовой выборке
- проверка лучшей модели на адекватность

В итоге лучшая модель это Логистическая регрессия с подобранными гиперпараметрами.
Лучшее значение метрики f1 = 0.76